In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import pandas as pd
import numpy as np
from collections import defaultdict

os.environ['CUDA_VISIBLE_DEVICES']='4'
pd.set_option('display.max_colwidth', None)
sys.path.append('/workspace/computationally-undermining-arguments/scripts/')
sys.path.append('/workspace/computationally-undermining-arguments/src-py')
sys.path.append('/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/')

In [3]:
%autoreload
import interact
import utility
import json
from argparse import Namespace
from tabulate import tabulate

In [4]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [5]:
jo_df = pd.read_pickle("/workspace/ceph_data/argument-undermining/jo_data/gen_data.pickle")
test_df = jo_df[jo_df.split=='test']
valid_df  = jo_df[jo_df.split=='val']

In [27]:
test_df['abortion'] = test_df.title.apply(lambda x: 'Abortion' in x or 'abortion' in x)
test_df['short_topic'] = test_df.title.apply(lambda x: len(x.split(' ')) < 3)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
test_df[test_df.short_topic][['title', 'post', 'premise_counter_premise_pairs']].head()

title  \
91004               
99370               
101235   feminism   
101236   feminism   
103208   Abortion   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [7]:
testing_df = test_df.sample(50)

In [8]:
testing_df['post'] = testing_df['post'].apply(lambda t: [x.strip().lower() for x in t])
testing_df['filtered_comment'] = testing_df.apply(lambda row: list(filter(lambda x: x.strip().lower() not in row['post'] or x.strip().lower() != row['title'].strip().lower(), row['comment_sents'])), axis=1)

In [9]:
#testing_df = pd.read_pickle('./testing_df.pkl')

In [10]:
args = Namespace(max_length=150, device='cuda:0', temperature=1.0, 
                 top_k=50, top_p=0.95, no_sample=False, min_length=100, premise_extra=False,
                 build_instance_version='v3'
)

baseline_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-48-03_gammaweb03_jo_baseline_gpt/'
app_v2_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-50-50_gammaweb03_my_approach_v2/'
app_v3_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-53-13_gammaweb03_my_approach_v3/'
app_v4_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec21_17-10-34_gammaweb03_my_approach_v4/'

In [11]:
def expand_df(df, pred_clms, baseline_preds_clms, attacks_clm='premise_counter_premise_pairs', post_clm='post', comment_clm='comment_sents'):
    expanded_data = []
    for idx, row in df.iterrows():
        for counter_idx, counter in enumerate(row[attacks_clm]):
            data = [
                row['title'],
                row[post_clm],
                row[comment_clm],
                ' '.join(row['filtered_comment']),
                #row['target'][counter_idx],
                counter[0],
                counter[1]
            ]
            
            for clm in baseline_preds_clms:
                data.append(row[clm])

            for clm in pred_clms:
                data.append(row[clm][counter_idx][1])
                
            expanded_data.append(data)

    expanded_df = pd.DataFrame(expanded_data, columns=['title', 'post', 'full_comment', 'filtered_comment', 'premise', 'gt_counter'] + baseline_preds_clms + pred_clms)
    
    return expanded_df

In [14]:
args.build_instance_version = 'v2'
args.temperatureerature = 0.7
args.max_length=100
args.min_length=50
testing_df = utility.pred_attacks(testing_df, app_v2_path, args, 'app_v2_0.7', context='title+full_post')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
Some weights of the model checkpoint at /workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-50-50_gammaweb03_my_approach_v2/ were not used when initializing OpenAIGPTLMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias', 'transformer.position_ids']
- This IS expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
args.build_instance_version = 'v2'
args.temperatureerature = 1.0
args.max_length=100
args.min_length=50
testing_df = utility.pred_attacks(testing_df, app_v2_path, args, 'app_v2_temp1.0', context='title+full_post')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
Some weights of the model checkpoint at /workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-50-50_gammaweb03_my_approach_v2/ were not used when initializing OpenAIGPTLMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias', 'transformer.position_ids']
- This IS expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
args.build_instance_version = 'v2'
args.temperatureerature = 0.3
args.max_length=100
args.min_length=50
testing_df = utility.pred_attacks(testing_df, app_v2_path, args, 'app_v2_0.3', context='title+full_post')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
Some weights of the model checkpoint at /workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-50-50_gammaweb03_my_approach_v2/ were not used when initializing OpenAIGPTLMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias', 'transformer.position_ids']
- This IS expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/interact.py:115: UserWarning: Warning: Weak premise coul

In [30]:
expanded_df = expand_df(testing_df, ['app_v2_0.7', 'app_v2_0.3', 'app_v2_temp1.0'], [])

In [31]:
expanded_df.head()

title  \
0      Women were not more oppressed during history than men were.   
1   Veterans aren't good people and should not be praised as such.   
2   Veterans aren't good people and should not be praised as such.   
3   Veterans aren't good people and should not be praised as such.   
4     white heterosexual women are the most privileged demographic   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  post  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      [for most of history, women have enjoyed being seen as the people that needed protection alongside kids and were sheltered ideally from danger., meanwhile, men had to work in shit conditions, died in wars, were expected to keep a family fed., and you might say ' uh yeah but women didn't have the choice to work! ! ', and that's true, but it's not like men had a choice whether to go to war., nor was being a 'houseman' a valid choice for most of history., what i'm getting at is that it's not men oppressing women, it was leaders oppressing plebs, both men and women, though here i argue that male plebs had it way worse.]   
1                                                                                                   

In [38]:
post_counter_premises_df = expanded_df[['title', 'gt_counter']].groupby('title').agg({
    'gt_counter' : lambda gt_counter: list(gt_counter)
}).reset_index()
post_counter_premises_dict = pd.Series(post_counter_premises_df.gt_counter.values, index=post_counter_premises_df.title).to_dict()

post_counter_arguments_df = expanded_df[['title', 'filtered_comment']].groupby('title').agg({
    'filtered_comment' : lambda filtered_comment: list(filtered_comment)
}).reset_index()
post_counter_arguments_dict = pd.Series(post_counter_arguments_df.filtered_comment.values, index=post_counter_arguments_df.title).to_dict()

In [39]:
expanded_df['all_counter_premises'] = expanded_df.title.apply(lambda x: post_counter_premises_dict[x])
expanded_df['all_counter_arguments'] = expanded_df.title.apply(lambda x: post_counter_arguments_dict[x])

------

In [40]:
testing_df.columns

Index(['post_id', 'split', 'comment_id', 'title', 'post', 'comment',
       'n_sentences', 'direct_sents', 'all_4_sents', 'comment_sents',
       'premise_counter_premise_pairs', 'num_attacks', 'filtered_comment',
       'pred_attack8', 'baseline_attack_2', 'weak_premises', 'pred_attack7',
       'pred_attack9', 'pred_attack10', 'pred_attack10_1epoch',
       'pred_attack10_6epochs', 'pred_attack09_6epochs', 'baseline_final',
       'app_v2', 'app_v3', 'rand_premises', 'app_v2_rand_0', 'app_v2_rand_1',
       'app_v2_rand_2', 'app_v1_rand_0', 'app_v1_rand_1', 'app_v1_rand_2',
       'app_v3_rand_0', 'app_v3_rand_1', 'app_v3_rand_2', 'app_v2_title',
       'app_v3_title', 'app_v4_rand_0', 'app_v4_rand_1', 'app_v4_rand_2',
       'app_v4_title', 'app_v4'],
      dtype='object')

In [41]:
testing_df.to_pickle('./testing_df.pkl')

In [42]:
expanded_df.to_pickle('./expanded_df.pkl')

In [43]:
expanded_df.columns

Index(['title', 'post', 'full_comment', 'filtered_comment', 'premise',
       'gt_counter', 'app_v2', 'app_v3', 'app_v4', 'baseline_final',
       'app_v3_rand_0', 'app_v3_rand_1', 'app_v3_rand_2', 'app_v2_rand_0',
       'app_v2_rand_1', 'app_v2_rand_2', 'app_v4_rand_0', 'app_v4_rand_1',
       'app_v4_rand_2', 'app_v2_title', 'app_v3_title', 'app_v4_title',
       'all_counter_premises', 'all_counter_arguments'],
      dtype='object')

In [114]:
expanded_df.head()

,title,post,full_comment,filtered_comment,premise,gt_counter,app_v2,app_v3,baseline_final,app_v1_rand_0,app_v1_rand_1,app_v1_rand_2,app_v2_rand_0,app_v2_rand_1,app_v2_rand_2,all_counter_premises,all_counter_arguments
0,"""God is good."" doesn't mean or say anything. Also, that sentence, when taken to its logical conclusion in the context of the real world and the Bible, implies worshippers are hostages suffering Stockolm syndrome.","[obligatory 'this probably applies to other religions but i'm focusing on christianity because i know more about it. ', like many agnostic atheists, one of my qualms with many god claims is the problem of evil., i, personally, think it's an extremely good argument against a tri omni god., usually, i find the response is questioning my perception of 'good' and saying 'good' is whatever god says it is., if you agree with this, my point is, this does not connect at all to our perception of good., if it did, god would not allow insert horrible event that happens daily in real life and encourage insert horrible bible verse about slavery women war punishment ., i'll be more specific if you actually want to address this last point, i more just typed it to illustrate my perception that if god is good, and wanted people to use the human word 'good' or it's translations to describe him, he would have actually wanted to imply that it describes him., also if you do not think his sense of 'good' is the same as yours, there is no reason to accept him as the definition other than here comes the hostage comparison because he's more powerful and says he'll punish us i know not all denominations agree on hell, but if you do this point is for you if we don't., also, it does not make sense to say god would use the word 'good' to describe himself, including translated versions of the word using the other language's version of that word, but not use the human understanding of it without clarifying what he means.]","[if you agree with this, my point is, this does not connect at all to our perception of good., if it did, god would not allow insert horrible event that happens daily in real life and encourage insert horrible bible verse about slavery women war punishment ., one of the common answers to this is that god has a logically sufficient reason to allow these all these horrible things, but that we as mere lowly humans in our current state are just unable to comprehend., it will all become clear in the afterlife., in other words, there is no logical incompatibility between the existence of evil and the existence of god, depending on your theology., this is not my view btw., i'm an atheist as well.]","if you agree with this, my point is, this does not connect at all to our perception of good. if it did, god would not allow insert horrible event that happens daily in real life and encourage insert horrible bible verse about slavery women war punishment . one of the common answers to this is that god has a logically sufficient reason to allow these all these horrible things, but that we as mere lowly humans in our current state are just unable to comprehend. it will all become clear in the afterlife. in other words, there is no logical incompatibility between the existence of evil and the existence of god, depending on your theology. this is not my view btw. i'm an atheist as well.","[if you agree with this, my point is, this does not connect at all to our perception of good., if it did, god would not allow insert horrible event that happens daily in real life and encourage insert horrible bible verse about slavery women war punishment .]","one of the common answers to this is that god has a logically sufficient reason to allow these all these horrible things, but that we as mere lowly humans in our current state are just unable to comprehend. it will all become clear in the afterlife. in other words, there is no logical incompatibility between the existence of evil and the existence of god, depending on your theology. 